In [1]:
from mongoengine import Document, StringField, IntField, EmailField, DateTimeField, BooleanField, ListField, EmbeddedDocument, EmbeddedDocumentField
import mongoengine as me
import datetime


class Address(EmbeddedDocument):
    street = StringField(required=True, max_length=200)
    city = StringField(required=True, max_length=100)
    state = StringField(max_length=2, regex=r'^[A-Z]{2}$')  # State code, e.g., NY, CA
    zip_code = StringField(required=True, max_length=10, regex=r'^\d{5}(-\d{4})?$')


class Customer(Document):
    first_name = StringField(required=True, max_length=50)
    last_name = StringField(required=True, max_length=50)
    email = EmailField(required=True, unique=True)
    age = IntField(min_value=18, max_value=100)
    created_at = DateTimeField(required=True, default=datetime.datetime.now)
    active = BooleanField(default=True)
    roles = ListField(StringField(choices=('admin', 'user', 'guest')))
    address = EmbeddedDocumentField(Address)
    
    meta = {
        'collection': 'customers',  # Custom collection name
        'indexes': [
            'email',  # Simple index
            'last_name',  # Another simple index
            {'fields': ('first_name', 'last_name'), 'unique': True},  # Compound index
        ],
        'auto_create_index': True,  # Automatically create indexes defined above
        'auto_create_index_on_save': True,
        'ordering': ['-created_at'],  # Default ordering
        'strict': False,  # Allow fields not defined in the schema
    }

In [3]:
# Connect to MongoDB
me.disconnect()
db = me.connect(host='mongodb://mongodb:27017/mongoengine')
db.drop_database('mongoengine')

# Create a new customer
john_doe = Customer(
    first_name="John",
    last_name="Doe",
    email="john.doe@example.com",
    age=18,
    roles=["admin", "user"],
    address=Address(
        street="123 Main St",
        city="Anytown",
        state="NY",
        zip_code="12345"
    )
)

# Save the customer to the database
john_doe.save()

<Customer: Customer object>

In [4]:
print(john_doe.to_json(indent=4))

{
    "_id": {
        "$oid": "666a2cb85a269c931e53382e"
    },
    "first_name": "John",
    "last_name": "Doe",
    "email": "john.doe@example.com",
    "age": 18,
    "created_at": {
        "$date": 1718234296565
    },
    "active": true,
    "roles": [
        "admin",
        "user"
    ],
    "address": {
        "street": "123 Main St",
        "city": "Anytown",
        "state": "NY",
        "zip_code": "12345"
    }
}


In [5]:
jane = Customer.objects.create(
    first_name="Jane",
    last_name="Smith",
    email="jane.smith@example.com",
    age=18,
    roles=["user"],
    address=Address(
        street="123 Main St",
        city="New York",
        state="NY",
        zip_code="12345"
    )
)
jane.to_json()

'{"_id": {"$oid": "666a2ce35a269c931e53382f"}, "first_name": "Jane", "last_name": "Smith", "email": "jane.smith@example.com", "age": 18, "created_at": {"$date": 1718234339858}, "active": true, "roles": ["user"], "address": {"street": "123 Main St", "city": "New York", "state": "NY", "zip_code": "12345"}}'

In [8]:
results = Customer.objects.filter(address__city="Anytown")
results

[<Customer: Customer object>]

In [9]:
print(results.to_json(indent=4))

[
    {
        "_id": {
            "$oid": "666a2cb85a269c931e53382e"
        },
        "first_name": "John",
        "last_name": "Doe",
        "email": "john.doe@example.com",
        "age": 18,
        "created_at": {
            "$date": 1718234296565
        },
        "active": true,
        "roles": [
            "admin",
            "user"
        ],
        "address": {
            "street": "123 Main St",
            "city": "Anytown",
            "state": "NY",
            "zip_code": "12345"
        }
    }
]


## Relationships between models

In [10]:
from mongoengine import ReferenceField


# Define the Post model --- CONSTRAINTS ARE ENFORCED BY MONGOENGINE, NOT BY PYMONGO
class Post(Document):
    title = StringField(required=True, min_length=1, max_length=200)
    content = StringField(required=True, min_length=1)
    
    author = ReferenceField(Customer, required=True)


# Ensure indexes are created
Post.ensure_indexes()

In [11]:
Post.objects.create(
    title="Example title",
    content="TBD",
    author=jane)

<Post: Post object>

In [12]:
print(jane.id)
print(Post.objects.first().to_json(indent=4))

666a2ce35a269c931e53382f
{
    "_id": {
        "$oid": "666a2d285a269c931e533830"
    },
    "title": "Example title",
    "content": "TBD",
    "author": {
        "$oid": "666a2ce35a269c931e53382f"
    }
}


# Pymongo

Let's say you need to remove some fields from a customer object. In mongo, you can do that with `$unset`, but you don't have that feature in mongoengine. How to proceed?

In [13]:
import pymongo
from bson import ObjectId

# Establish a connection to MongoDB using PyMongo
client = pymongo.MongoClient('mongodb://mongodb:27017/')
db = client['mongoengine']

# Convert the MongoEngine document to its "pymongo" form
customer_mongo = john_doe.to_mongo()
print(customer_mongo)

SON([('_id', ObjectId('666a2cb85a269c931e53382e')), ('first_name', 'John'), ('last_name', 'Doe'), ('email', 'john.doe@example.com'), ('age', 18), ('created_at', datetime.datetime(2024, 6, 12, 23, 18, 16, 565545)), ('active', True), ('roles', ['admin', 'user']), ('address', SON([('street', '123 Main St'), ('city', 'Anytown'), ('state', 'NY'), ('zip_code', '12345')]))])


In [14]:
customer_mongo.to_dict()

{'_id': ObjectId('666a2cb85a269c931e53382e'),
 'first_name': 'John',
 'last_name': 'Doe',
 'email': 'john.doe@example.com',
 'age': 18,
 'created_at': datetime.datetime(2024, 6, 12, 23, 18, 16, 565545),
 'active': True,
 'roles': ['admin', 'user'],
 'address': {'street': '123 Main St',
  'city': 'Anytown',
  'state': 'NY',
  'zip_code': '12345'}}

In [15]:
# Now, use PyMongo to perform an advanced operation such as removing the 'age' field
db.customers.update_one({'_id': customer_mongo['_id']}, {'$unset': {'age': ''}})

# Verify that the 'age' field has been removed
updated_customer = db.customers.find_one({'_id': ObjectId(customer_mongo['_id'])})
updated_customer  # Not a "Customer" model but a dict

{'_id': ObjectId('666a2cb85a269c931e53382e'),
 'first_name': 'John',
 'last_name': 'Doe',
 'email': 'john.doe@example.com',
 'created_at': datetime.datetime(2024, 6, 12, 23, 18, 16, 565000),
 'active': True,
 'roles': ['admin', 'user'],
 'address': {'street': '123 Main St',
  'city': 'Anytown',
  'state': 'NY',
  'zip_code': '12345'}}

In [16]:
# Perform an aggregation query
# For example, counting customers by state
pipeline = [
    {'$group': {'_id': '$address.state', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

results = db.customers.aggregate(pipeline)
for result in results:
    print(result)

{'_id': 'NY', 'count': 2}
